In [1]:
import os 
import numpy as np 
import matplotlib.pyplot as plt 
import cv2 
import pickle 

data_dir = './DATASET/train'

categories = ['angry', 'disgust' , 'fear', 'happy', 'neutral', 'sad', 'surprise']

data = []

def make_data():
	for category in categories:
		path = os.path.join(data_dir, category)
		label = categories.index(category)

		for img_name in os.listdir(path):
			image_path = os.path.join(path, img_name)
			image = cv2.imread(image_path)
			

			try:
				image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
				image = cv2.resize(image, (224,224))
				image = np.array(image)

				data.append([image, label])

			except Exception as e:
				pass

	print(len(data))
	pik = open('train.pickle','wb')
	pickle.dump(data, pik)
	pik.close()
make_data()
def load_data():
	pick  = open('train.pickle', 'rb')
	data = pickle.load(pick)

	pick.close()

	#np.random(data)

	feature = []
	labels = []

	for img, label in data:
		feature.append(img)
		labels.append(label)

	feature = np.array(feature, dtype = np.float32)
	feature = feature/ 255.

	labels = np.array(labels)

	return [feature, labels]


28709


In [2]:
import tensorflow as tf 


input_layer = tf.keras.layers.Input([224,224,3])
#block1 
conv1_1 = tf.keras.layers.Conv2D(filters=64, kernel_size=[3,3],strides=[1,1], 
	padding='same',activation='relu', name='conv1_1')(input_layer)

conv1_2 = tf.keras.layers.Conv2D(filters= 64, kernel_size=[3,3], strides= [1,1],
	padding='same',activation='relu', name='conv1_2')(conv1_1)

pool1_1 = tf.nn.max_pool(conv1_2, ksize = [1,2,2,1],strides=[1,2,2,1],
	padding='SAME', name='pool1_1')


#block 2
conv2_1 = tf.keras.layers.Conv2D(filters= 128, kernel_size=[3,3],strides=[1,1],
	padding='same', activation='relu', name='conv2_1')(pool1_1)


conv2_2 = tf.keras.layers.Conv2D(filters = 128, kernel_size=[3,3], strides = [1,1],
	padding='same', activation='relu', name='conv2_2')(conv2_1)


pool2_1 = tf.nn.max_pool(conv2_2, ksize = [1,2,2,1], strides= [1,2,2,1],
	padding='SAME', name='pool2_1')



#block3 

conv3_1 = tf.keras.layers.Conv2D(filters= 256, kernel_size=[3,3], strides = [1,1],
	padding='same', activation='relu', name='conv3_1')(pool2_1)

conv3_2 = tf.keras.layers.Conv2D(filters = 256, kernel_size=[3,3], strides=[1,1],
	padding='same', activation='relu', name='conv3_2')(conv3_1)

conv3_3 = tf.keras.layers.Conv2D(filters = 256, kernel_size= [3,3], strides=[1,1],
	padding='same', activation='relu', name='conv3_3')(conv3_2)


conv3_4 = tf.keras.layers.Conv2D(filters = 256, kernel_size= [3,3], strides=[1,1],
	padding='same', activation='relu', name='conv3_4')(conv3_3)


pool3_1 = tf.nn.max_pool(conv3_4, ksize=[1,2,2,1], strides=[1,2,2,1],
	padding="SAME",name='pool3_1')

#block4 

conv4_1 = tf.keras.layers.Conv2D(filters = 512, kernel_size= [3,3], strides=[1,1],
	padding='same', activation='relu', name='conv4_1')(pool3_1)


conv4_2 = tf.keras.layers.Conv2D(filters = 512, kernel_size= [3,3], strides=[1,1],
	padding='same', activation='relu', name='conv4_2')(conv4_1)


conv4_3 = tf.keras.layers.Conv2D(filters = 512, kernel_size= [3,3], strides=[1,1],
	padding='same', activation='relu', name='conv4_3')(conv4_2)


conv4_4 = tf.keras.layers.Conv2D(filters = 512, kernel_size= [3,3], strides=[1,1],
	padding='same', activation='relu', name='conv4_4')(conv4_3)

pool4_1 = tf.nn.max_pool(conv4_4, ksize=[1,2,2,1],strides=[1,2,2,1],
	padding='SAME', name='pool4_1')

#block5


conv5_1 = tf.keras.layers.Conv2D(filters = 512, kernel_size= [3,3], strides=[1,1],
	padding='same', activation='relu', name='conv5_1')(pool4_1)

conv5_2 = tf.keras.layers.Conv2D(filters = 512, kernel_size= [3,3], strides=[1,1],
	padding='same', activation='relu', name='conv5_2')(conv5_1)

conv5_3 = tf.keras.layers.Conv2D(filters = 512, kernel_size= [3,3], strides=[1,1],
	padding='same', activation='relu', name='conv5_3')(conv5_2)

conv5_4 = tf.keras.layers.Conv2D(filters = 512, kernel_size= [3,3], strides=[1,1],
	padding='same', activation='relu', name='conv5_4')(conv5_3)

pool5_1 = tf.nn.max_pool(conv5_4, ksize=[1,2,2,1], strides=[1,2,2,1],
	padding='SAME', name='pool5_1')



flatten  = tf.keras.layers.Flatten()(pool5_1)

fc6 = tf.keras.layers.Dense(units=256, name='fc6', activation='relu')(flatten)
fc7 = tf.keras.layers.Dense(units=128, name='fc7', activation='relu')(fc6)
fc8 = tf.keras.layers.Dense(units=5, name='fc8',activation=None)(fc7)


prob = tf.nn.softmax(fc8)



model = tf.keras.Model(input_layer, prob)
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_1 (Conv2D)            (None, 224, 224, 64)      1792      
                                                                 
 conv1_2 (Conv2D)            (None, 224, 224, 64)      36928     
                                                                 
 tf.nn.max_pool2d (TFOpLambd  (None, 112, 112, 64)     0         
 a)                                                              
                                                                 
 conv2_1 (Conv2D)            (None, 112, 112, 128)     73856     
                                                                 
 conv2_2 (Conv2D)            (None, 112, 112, 128)     147584    
                                                             

In [3]:
#from utils import load_data
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split  
import tensorflow as tf 

feature, label = load_data()
categories = ['angry', 'disgust', 'fear','happy', 'neutral', 'sad', 'surprise']

x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.1, shuffle=True)

x_train, x_test = tf.cast(x_train, tf.float32), tf.cast(x_test, tf.float32)

train_dataset= tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_dataset = train_dataset.batch(batch_size= 1)




model = tf.keras.models.load_model('myVggModel.h5')

loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adadelta()


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')


@tf.function
def train_step(images, labels):
	with tf.GradientTape() as tape:
		predictions = model(images, training= True)
		loss = loss_object(y_true= labels, y_pred= predictions)

	gradients = tape.gradient(loss, model.trainable_variables)
	optimizer.apply_gradients(grads_and_vars = zip(gradients, model.trainable_variables))

	train_loss(loss)
	train_accuracy(labels, predictions)



for epoch in range(1):
	train_loss.reset_states()
	train_accuracy.reset_states()

	step = 0

	for images, labels in train_dataset:
		step+=1 
		train_step(images, labels)

		if step%10 ==0:

			print('=> epoch: %i, loss: %.4f, train_accuracy: %.4f'%(epoch+1,
				train_loss.result(), train_accuracy.result()))
model.save('myVggModel.h5')


MemoryError: Unable to allocate 16.1 GiB for an array with shape (28709, 224, 224, 3) and data type float32

In [4]:
#from utils import load_data
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split  
import tensorflow as tf 
import numpy as np 

feature, label = load_data()
x_train, x_test, y_train, y_test = train_test_split(feature, label, test_size=0.1, shuffle=True)

categories = ['angry', 'disgust', 'fear','happy', 'neutral', 'sad', 'surprise']

model = tf.keras.models.load_model('myVggModel.h5')


prediction = model(x_test[0:9])

plt.figure(figsize=(8,8))

for i in range(9):
	plt.subplot(3,3,i+1)
	plt.imshow(x_test[i])
	plt.xlabel('Pedicted:%s\n Actual: %s'%(categories[np.argmax(prediction[i])],
		categories[y_test[i]]))

	plt.xticks([])

plt.show()

MemoryError: Unable to allocate 16.1 GiB for an array with shape (28709, 224, 224, 3) and data type float32